# Índice
1. Preparação
 1. [Instalação de alguns pacotes](#Instalação-de-alguns-pacotes)
 2. [Definição de algumas configurações](#Definição-de-algumas-configurações)
 3. [Definição de algumas variáveis](#Definição-de-algumas-variáveis)
2. Inspeção dos dados
 4. [Dados brutos](#Dados-brutos)
 5. [Inserção de dados no bancos de dados](#Inserção-de-dados-no-bancos-de-dados)
 6. [Tabelas disponíveis no banco da dados](#Tabelas-disponíveis-no-banco-da-dados)
 7. [Número de transações](#Número-de-transações)
 8. [Nomenclatura Comum do Mercosul (NCM)](#Nomenclatura-Comum-do-Mercosul-(NCM))
 9. [Unidades Federativas (UF)](#Unidades-Federativas-(UF))
 10. [Informações gerais sobre a tabela de transações](#Informações-gerais-sobre-a-tabela-de-transações)
 11. [Quantidade de valores distintos por coluna](#Quantidade-de-valores-distintos-por-coluna)
3. Estatísticas
 12. [Transações com maior valor FOB (Free on Board) em dólar](#Transações-com-maior-valor-FOB-(Free-on-Board\)-em-dólar)
 13. [Mercadorias com maior valor FOB total em dólar](#Mercadorias-com-maior-valor-FOB-total-em-dólar)
 14. [Estados com maior valor FOB total em dólar](#Estados-com-maior-valor-FOB-total-em-dólar)
 15. [Valor FOB total anual em dólar](#Valor-FOB-total-anual-em-dólar)
 16. [Mercadorias com maior valor FOB total em dólar, em São Paulo, em 2019](#Mercadorias-com-maior-valor-FOB-total-em-dólar,-em-São-Paulo,-em-2019)
 17. [Valor FOB total em dólar de transações de cada produto, por estado, a cada ano](#Valor-FOB-total-em-dólar-de-transações-de-cada-produto,-por-estado,-a-cada-ano)
 18. [Produtos com o maior valor FOB total, por ano, por estado](#Produtos-com-o-maior-valor-FOB-total,-por-ano,-por-estado)
 19. [Valor FOB total em dólar de transações de cada produto, por estado, em cada mês de 2019](#Valor-FOB-total-em-dólar-de-transações-de-cada-produto,-por-estado,-em-cada-mês-de-2019)
 20. [Produtos com o maior valor FOB total em 2019, por mês, por estado](#Produtos-com-o-maior-valor-FOB-total-em-2019,-por-mês,-por-estado)
 21. [Posição de cada produto no ranking de valor transacionado por estado, por ano](#Posição-de-cada-produto-no-ranking-de-valor-transacionado-por-estado,-por-ano)
 22. **[Top 3 produtos com maior valor transacionado de 2017 a 2019, por ano, por estado](#Top-3-produtos-com-maior-valor-transacionado-de-2017-a-2019,-por-ano,-por-estado)**
 23. **[Top 3 produtos com maior valor transacionado em 2019, por mês, por estado](#Top-3-produtos-com-maior-valor-transacionado-em-2019,-por-mês,-por-estado)**
 24. [Valor FOB total transacionado em 2019](#Valor-FOB-total-transacionado-em-2019)
 25. **[Percentual com que cada estado contribuiu para o total transacionado pelo Brasil em 2019](#Percentual-com-que-cada-estado-contribuiu-para-o-total-transacionado-pelo-Brasil-em-2019)**

## Instalação de alguns pacotes

In [1]:
import pandas as pd
import sys
from glob import glob
from IPython.core.display import HTML
from sqlalchemy import create_engine

## Definição de algumas configurações

In [2]:
# Make better use of the space avaiable on screen
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 300)
display(HTML("<style>.container { width:100% !important; } div.prompt {min-width: 10ex;} .rendered_html table { font-size: 11px; }</style>"))

## Definição de algumas variáveis

In [3]:
# Paths for all the files used
path = '../data'
files = {'import': path + '/IMP_*.csv',
         'export': path + '/EXP_*.csv'}
ncm_file = path + '/NCM.csv'
uf_file = path + '/UF.csv'
db_file = path + '/trades.db'

# Kind of trade which will be used during the exploration (the other case is analogous)
kind = 'import'

# Connection to SQLite database
eng = create_engine(f'sqlite:///{db_file}')
conn = eng.connect()

## Dados brutos

Estas são as primeiras linhas do primeiro arquivo CSV do tipo (`kind`) de transação de mercadorias escolhido (`import` ou `export`):

In [4]:
%%timeit -n1 -r1
!head -n 4 {files[kind]}

==> ../data/IMP_2017.csv <==
"CO_ANO";"CO_MES";"CO_NCM";"CO_UNID";"CO_PAIS";"SG_UF_NCM";"CO_VIA";"CO_URF";"QT_ESTAT";"KG_LIQUIDO";"VL_FOB"
"2017";"01";"54024400";"10";"399";"SP";"01";"0817800";4770;4770;47344
"2017";"01";"84834090";"11";"386";"SP";"01";"0817800";13055;48308;468136
"2017";"01";"84719014";"11";"399";"SP";"04";"0817700";3;6;146

==> ../data/IMP_2018.csv <==
"CO_ANO";"CO_MES";"CO_NCM";"CO_UNID";"CO_PAIS";"SG_UF_NCM";"CO_VIA";"CO_URF";"QT_ESTAT";"KG_LIQUIDO";"VL_FOB"
"2018";"01";"85423190";"11";"160";"RS";"04";"0817700";139;1;261
"2018";"01";"85334011";"11";"399";"RS";"04";"1017801";1000;0;230
"2018";"02";"85049090";"10";"160";"PR";"04";"0917900";44;44;30066

==> ../data/IMP_2019.csv <==
"CO_ANO";"CO_MES";"CO_NCM";"CO_UNID";"CO_PAIS";"SG_UF_NCM";"CO_VIA";"CO_URF";"QT_ESTAT";"KG_LIQUIDO";"VL_FOB"
"2019";"12";"85044021";"11";"249";"SP";"04";"0817700";53;44;8263
"2019";"12";"85183000";"11";"160";"GO";"04";"0817600";23734;1621;6987
"2019";"12";"39263000";"10";"023";"SP";"01";"0

## Inserção de dados no bancos de dados

 Caso o banco de dados ainda não exista, ou esteja vazio, pode-se adicionar os dados usando o código a seguir:

In [5]:
# %%timeit -n1 -r1
# chunk_size=500000
# for kind_of_trade, pattern in files.items():
#     for name in glob(pattern):
#         print(name)
#         for chunk in pd.read_csv(name, delimiter=';', chunksize=chunk_size):
#             chunk.to_sql(kind_of_trade, eng, if_exists='append')

In [6]:
# %%timeit -n1 -r1
# for chunk in pd.read_csv(ncm_file, delimiter=';', chunksize=chunk_size, encoding = 'ISO-8859-1'):
#     chunk.to_sql('ncm', eng, if_exists='append')

In [7]:
# %%timeit -n1 -r1
# for chunk in pd.read_csv(uf_file, delimiter=';', chunksize=chunk_size, encoding = 'ISO-8859-1'):
#     chunk.to_sql('uf', eng, if_exists='append')

## Tabelas disponíveis no banco da dados

In [8]:
pd.read_sql("SELECT * FROM sqlite_master WHERE type='table'", conn, index_col=["name"])

,type,tbl_name,rootpage,sql
name,,,,
import,table,import,2,"CREATE TABLE import (\n\t""CO_NCM"" BIGINT, \n\t""CO_UNID"" BIGINT, \n\t""CO_PAIS"" BIGINT, \n\t""SG_UF_NCM"" TEXT, \n\t""CO_VIA"" BIGINT, \n\t""CO_URF"" BIGINT, \n\t""QT_ESTAT"" BIGINT, \n\t""KG_LIQUIDO"" BIGINT, \n\t""VL_FOB"" BIGINT, \n\t""DATA"" DATETIME\n)"
export,table,export,83896,"CREATE TABLE export (\n\t""CO_NCM"" BIGINT, \n\t""CO_UNID"" BIGINT, \n\t""CO_PAIS"" BIGINT, \n\t""SG_UF_NCM"" TEXT, \n\t""CO_VIA"" BIGINT, \n\t""CO_URF"" BIGINT, \n\t""QT_ESTAT"" BIGINT, \n\t""KG_LIQUIDO"" BIGINT, \n\t""VL_FOB"" BIGINT, \n\t""DATA"" DATETIME\n)"
ncm,table,ncm,138860,"CREATE TABLE ncm (\n\t""CO_NCM"" BIGINT, \n\t""CO_UNID"" BIGINT, \n\t""CO_SH6"" BIGINT, \n\t""CO_PPE"" BIGINT, \n\t""CO_PPI"" BIGINT, \n\t""CO_FAT_AGREG"" BIGINT, \n\t""CO_CUCI_ITEM"" TEXT, \n\t""CO_CGCE_N3"" BIGINT, \n\t""CO_SIIT"" BIGINT, \n\t""CO_ISIC_CLASSE"" BIGINT, \n\t""CO_EXP_SUBSET"" FLOAT, \n\t""NO_NCM_POR"" ..."
uf,table,uf,139522,"CREATE TABLE uf (\n\t""CO_UF"" BIGINT, \n\t""SG_UF"" TEXT, \n\t""NO_UF"" TEXT, \n\t""NO_REGIAO"" TEXT\n)"


## Número de transações

Contagem do número de linhas dos arquivos de transações de mercadorias. **Obs**: esse total inclui a primeira linha de cada arquivo (nomes das colunas)

In [9]:
%%timeit -n1 -r1
!wc -l {files[kind]}

  1823962 ../data/IMP_2017.csv
  1866512 ../data/IMP_2018.csv
  1904414 ../data/IMP_2019.csv
  5594888 total
1.58 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Aqui está o verdadeiro número de linhas na tabela recém criada:

In [10]:
%%timeit -n1 -r1
# conn.execute(f'SELECT COUNT(*) FROM {kind}').fetchall()
display(pd.read_sql(f'SELECT COUNT(*) FROM {kind}', conn))

,COUNT(*)
0,5594885


478 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Nomenclatura Comum do Mercosul (NCM)

In [11]:
%%timeit -n1 -r1
!wc -l {ncm_file}

13127 ../data/NCM.csv
123 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


O número de linhas na tabela recém criada é mostrado a seguir:

In [12]:
%%timeit -n1 -r1
# conn.execute('SELECT COUNT(*) FROM ncm').fetchall()
display(pd.read_sql('SELECT COUNT(*) FROM ncm', conn))

,COUNT(*)
0,13117


25.4 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


O comando `wc -l` retorna 10 linhas a mais pois, além de considerar a primeira linha, considera as 9 quebras de linha (`\n`) em células contendo strings:

In [13]:
%%timeit -n1 -r1
df = pd.read_sql('SELECT * FROM ncm', conn)
display(df[df.apply(lambda row: row.str.contains('\n').any(), axis=1)])

,CO_NCM,CO_UNID,CO_SH6,CO_PPE,CO_PPI,CO_FAT_AGREG,CO_CUCI_ITEM,CO_CGCE_N3,CO_SIIT,CO_ISIC_CLASSE,CO_EXP_SUBSET,NO_NCM_POR,NO_NCM_ESP,NO_NCM_ING
12731,85235210,11,852352,3235,3235,3,89846,240,1000,2610,1106.0,\t\r\nCartões e etiquetas de acionamento por aproximação,Tarjetas y etiquetas de activación por proximidad,Proximity cards and proximity tags
12734,27075010,10,270750,3215,3990,3,33525,240,2000,2011,8099.0,"Misturas que contenham trimetilbenzenos e etiltoluenos, como componentes majoritários","\r\nMezclas que contengan trimetilbenzenos y etiltoluenos, como componentes mayoritarios",\r\nMixtures containing trimethylbenzenes and ethyltoluenes as major constituents
12735,54033110,10,540331,3275,3275,3,65174,240,2000,2030,1601.0,"Fio de raiom crus ou branqueados viscose, sem torção ou com torção não superior a 120 voltas por metro","Hiilado de rayón viscosa \r\r\ncrudos o blanqueados, sencillo, torsión<=120vueltas/m","Yarn of viscose rayon \r\r\ncrushed or bleached, single, twist<=120turns/meter"
12740,95089043,10,950890,3070,3070,3,8946,110,2000,2829,8099.0,\t\r\n\r\nEquipamentos recreativos para parques aquáticos \t\r\n\r\nEquipamentos recreativos para parques aquáticos,Equipamiento recreativo para parques acuáticos.,Recreational equipment for water parks


3.27 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Unidades Federativas (UF)

In [14]:
%%timeit -n1 -r1
!wc -l {uf_file}

35 ../data/UF.csv
112 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [15]:
%%timeit -n1 -r1
# conn.execute('SELECT COUNT(*) FROM uf').fetchall()
display(pd.read_sql('SELECT COUNT(*) FROM uf', conn))

,COUNT(*)
0,34


6.12 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [16]:
display(pd.read_sql('SELECT * FROM uf LIMIT 5', conn))

,CO_UF,SG_UF,NO_UF,NO_REGIAO
0,11,RO,Rondônia,REGIAO NORTE
1,12,AC,Acre,REGIAO NORTE
2,13,AM,Amazonas,REGIAO NORTE
3,14,RR,Roraima,REGIAO NORTE
4,15,PA,Pará,REGIAO NORTE


## Informações gerais sobre a tabela de transações

In [17]:
# Disabled timing due to NameError. See https://github.com/ipython/ipython/issues/11659/
# %%timeit -n1 -r1
# 24.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
df = pd.read_sql(f'SELECT * FROM {kind}', conn)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5594885 entries, 0 to 5594884
Data columns (total 10 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   CO_NCM      int64 
 1   CO_UNID     int64 
 2   CO_PAIS     int64 
 3   SG_UF_NCM   object
 4   CO_VIA      int64 
 5   CO_URF      int64 
 6   QT_ESTAT    int64 
 7   KG_LIQUIDO  int64 
 8   VL_FOB      int64 
 9   DATA        object
dtypes: int64(8), object(2)
memory usage: 426.9+ MB


## Quantidade de valores distintos por coluna

In [18]:
%%timeit -n1 -r1
display(df.nunique())

CO_NCM          9166
CO_UNID           13
CO_PAIS          250
SG_UF_NCM         28
CO_VIA            10
CO_URF           132
QT_ESTAT      227247
KG_LIQUIDO    181475
VL_FOB        447867
DATA              36
dtype: int64

901 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Transações com maior valor FOB (Free on Board) em dólar

In [19]:
%%timeit -n1 -r1
# conn.execute(f'SELECT * FROM {kind} ORDER BY VL_FOB DESC LIMIT 5').fetchall()
display(pd.read_sql(f'SELECT * FROM {kind} ORDER BY VL_FOB DESC LIMIT 5', conn))

,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB,DATA
0,89052000,11,105,RJ,10,917800,0,81451179,2215563252,2018-11-01 00:00:00.000000
1,89052000,11,160,RJ,10,1017700,0,101748000,2085171726,2019-01-01 00:00:00.000000
2,89052000,11,160,RJ,10,1017700,0,101748000,2071124563,2018-08-01 00:00:00.000000
3,89059000,11,105,RJ,10,1017700,2,90511000,1985998896,2018-02-01 00:00:00.000000
4,89052000,11,105,RJ,10,727600,2,76224380,1709212920,2019-09-01 00:00:00.000000


1.74 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Mercadorias com maior valor FOB total em dólar

Os maiores valores FOB totais, por mercadoria são os seguintes:

In [20]:
%%timeit -n1 -r1
query = f'''
SELECT CO_NCM,
       SUM(VL_FOB) total
FROM {kind}
GROUP BY CO_NCM
ORDER BY total DESC
LIMIT 3
'''
display(pd.read_sql(query, conn))

,CO_NCM,total
0,27101921,18603831959
1,27090010,12664164160
2,27101241,9952530812


2.96 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Os nomes das mercadorias correspondentes aos códigos da coluna CO_NCM acima podem ser obtidos da tabela `ncm`:

In [21]:
%%timeit -n1 -r1
query = f'''
SELECT k.CO_NCM,
       NO_NCM_POR,
       SUM(VL_FOB) total
FROM {kind} k
JOIN ncm ON ncm.CO_NCM = k.CO_NCM
GROUP BY k.CO_NCM
ORDER BY total DESC
LIMIT 3
'''
display(pd.read_sql(query, conn))

,CO_NCM,NO_NCM_POR,total
0,27101921,Gasóleo (óleo diesel),18603831959
1,27090010,Óleos brutos de petróleo,12664164160
2,27101241,Naftas para petroquimica,9952530812


6.47 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Estados com maior valor FOB total em dólar

In [22]:
%%timeit -n1 -r1
query = f'''
SELECT k.SG_UF_NCM,
       uf.NO_UF,
       SUM(VL_FOB) total
FROM {kind} k
JOIN uf ON uf.SG_UF = k.SG_UF_NCM
GROUP BY k.SG_UF_NCM
ORDER BY total DESC
LIMIT 3
'''
display(pd.read_sql(query, conn))

,SG_UF_NCM,NO_UF,total
0,SP,São Paulo,175421880962
1,RJ,Rio de Janeiro,56304101371
2,SC,Santa Catarina,44973206316


3.57 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Valor FOB total anual em dólar

In [23]:
%%timeit -n1 -r1
query = f'''
SELECT strftime("%Y", DATA) ano,
       SUM(VL_FOB) total
FROM {kind}
GROUP BY ano
ORDER BY ano
'''
display(pd.read_sql(query, conn))

,ano,total
0,2017,150749494421
1,2018,181230568862
2,2019,177347934749


3.04 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Mercadorias com maior valor FOB total em dólar, em São Paulo, em 2019

In [24]:
%%timeit -n1 -r1
year=2019
state='SP'
query = f'''
SELECT strftime("%Y", k.DATA) ano,
       u.NO_UF,
       n.NO_NCM_POR,
       SUM(VL_FOB) total
FROM {kind} k
JOIN ncm n ON k.CO_NCM=n.CO_NCM
JOIN uf u ON SG_UF_NCM=u.SG_UF
WHERE ano="{year}"
  AND SG_UF_NCM="{state}"
GROUP BY k.CO_NCM
ORDER BY total DESC
LIMIT 3
'''
display(pd.read_sql(query, conn))

,ano,NO_UF,NO_NCM_POR,total
0,2019,São Paulo,Gasóleo (óleo diesel),1627159797
1,2019,São Paulo,Outras partes para aparelhos de telefonia/telegrafia,1548336237
2,2019,São Paulo,Óleos brutos de petróleo,1071774913


2.17 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


##  Valor FOB total em dólar de transações de cada produto, por estado, a cada ano

In [25]:
%%timeit -n1 -r1
query = f'''
SELECT u.NO_UF estado,
       strftime("%Y", k.DATA) ano,
       n.NO_NCM_POR produto,
       SUM(VL_FOB) total
FROM {kind} k
JOIN ncm n ON k.CO_NCM=n.CO_NCM
JOIN uf u ON SG_UF_NCM=u.SG_UF
GROUP BY ano,
         estado,
         produto
ORDER BY ano,
         estado,
         total DESC
'''
display(pd.read_sql(query, conn))

,estado,ano,produto,total
0,Acre,2017,"Alhos, frescos ou refrigerados, exceto para semeadura",393150
1,Acre,2017,"Outros policloretos de vinila, em formas primárias",293144
2,Acre,2017,"Castanha-do-pará, fresca ou seca, sem casca",235580
3,Acre,2017,Sulfatos de cromo,168608
4,Acre,2017,Máquinas e aparelhos para preparação de carnes,123502
...,...,...,...,...
201624,Zona Não Declarada,2019,"Arruelas de pressão e outras arruelas de segurança, de ferro fundido, ferro ou aço",3
201625,Zona Não Declarada,2019,Outros próprios para aparelhos telefônicos,2
201626,Zona Não Declarada,2019,Outros isqueiros e acendedores,2
201627,Zona Não Declarada,2019,Outras máquinas-ferramentas para afiar metais/ceramais,2


14 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Produtos com o maior valor FOB total, por ano, por estado

In [26]:
%%timeit -n1 -r1
query = f'''
WITH group_totals (estado, ano, produto, total) AS
  (SELECT k.SG_UF_NCM estado,
          strftime("%Y", k.DATA) ano,
          k.CO_NCM produto,
          SUM(VL_FOB) total
   FROM {kind} k
   GROUP BY ano,
            estado,
            produto)
SELECT u.NO_UF estado,
       ano,
       n.NO_NCM_POR produto,
       MAX(total) max_total
FROM group_totals
JOIN ncm n ON produto=n.CO_NCM
JOIN uf u ON estado=u.SG_UF
GROUP BY estado,
         ano
ORDER BY estado,
         ano
'''
display(pd.read_sql(query, conn))

,estado,ano,produto,max_total
0,Acre,2017,"Alhos, frescos ou refrigerados, exceto para semeadura",393150
1,Acre,2018,"Revestimentos para pisos (pavimentos) e capachos, de borracha vulcanizada não endurecida",1031956
2,Acre,2019,"Pneumáticos novos, de borracha, dos tipos utilizados em automóveis de passageiros (incluindo os veículos de uso misto (station wagons) e os automóveis de corrida)",445790
3,Alagoas,2017,"Alhos, frescos ou refrigerados, exceto para semeadura",36706566
4,Alagoas,2018,"Alhos, frescos ou refrigerados, exceto para semeadura",23857871
...,...,...,...,...
79,Tocantins,2018,"Outras naftas, exceto para petroquímica",35627949
80,Tocantins,2019,Outros cloretos de potássio,45001029
81,Zona Não Declarada,2017,"Outros produtos imunológicos, apresentados em doses ou acondicionados para venda a retalho",11099796
82,Zona Não Declarada,2018,"Aviões e outros veículos aéreos, 7 toneladas < peso <= 15 toneladas, vazios, a turboélice",29215851


7.57 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


##  Valor FOB total em dólar de transações de cada produto, por estado, em cada mês de 2019

In [27]:
%%timeit -n1 -r1
year = 2019
query = f'''
SELECT u.NO_UF estado,
       strftime("%Y", k.DATA) ano,
       ltrim(strftime("%m", k.DATA), "0") mês,
       n.NO_NCM_POR produto,
       SUM(VL_FOB) total
FROM {kind} k
JOIN ncm n ON k.CO_NCM=n.CO_NCM
JOIN uf u ON SG_UF_NCM=u.SG_UF
WHERE ano = "{year}"
GROUP BY mês,
         estado,
         produto
ORDER BY mês,
         estado,
         total DESC
'''
display(pd.read_sql(query, conn))

,estado,ano,mês,produto,total
0,Acre,2019,1,"Pneumáticos novos, de borracha, dos tipos utilizados em automóveis de passageiros (incluindo os veículos de uso misto (station wagons) e os automóveis de corrida)",118213
1,Acre,2019,1,"Outros aparelhos elevadores ou transportadores, de ação contínua, para mercadorias",85945
2,Acre,2019,1,"Outros policloretos de vinila, em formas primárias",47423
3,Acre,2019,1,"Outros pneumáticos novos, de borracha",34035
4,Acre,2019,1,Pigmentos tipo rutilo,24493
...,...,...,...,...,...
406807,Zona Não Declarada,2019,9,Outros controladores eletrônicos automáticos para veículos automóveis,21
406808,Zona Não Declarada,2019,9,Outras obras de ferro ou aço,14
406809,Zona Não Declarada,2019,9,"Outros livros, brochuras e impressos semelhantes",8
406810,Zona Não Declarada,2019,9,Válvulas solenóides,5


7.18 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Produtos com o maior valor FOB total em 2019, por mês, por estado

In [28]:
%%timeit -n1 -r1
year = 2019
query = f'''
WITH group_totals (estado, ano, mês, produto, total) AS
  (SELECT SG_UF_NCM estado,
          strftime("%Y", k.DATA) ano,
          ltrim(strftime("%m", k.DATA), "0") mês,
          k.CO_NCM produto,
          SUM(VL_FOB) total
   FROM {kind} k
   JOIN ncm n ON k.CO_NCM=n.CO_NCM
   JOIN uf u ON SG_UF_NCM=u.SG_UF
   WHERE ano = "{year}"
   GROUP BY mês,
            estado,
            produto)
SELECT u.NO_UF estado,
       ano,
       mês,
       n.NO_NCM_POR produto,
       MAX(total) max_total
FROM group_totals
JOIN ncm n ON produto=n.CO_NCM
JOIN uf u ON estado=u.SG_UF
GROUP BY estado,
         mês
ORDER BY estado,
         mês
'''
display(pd.read_sql(query, conn))

,estado,ano,mês,produto,max_total
0,Acre,2019,1,"Pneumáticos novos, de borracha, dos tipos utilizados em automóveis de passageiros (incluindo os veículos de uso misto (station wagons) e os automóveis de corrida)",118213
1,Acre,2019,10,"Outros produtos imunológicos, apresentados em doses ou acondicionados para venda a retalho",43399
2,Acre,2019,11,Sulfatos de cromo,33436
3,Acre,2019,12,"Outros policloretos de vinila, em formas primárias",49698
4,Acre,2019,2,"Outros pneumáticos novos, de borracha, dos tipos utilizados em ônibus ou caminhões",50118
...,...,...,...,...,...
331,Zona Não Declarada,2019,5,"Aviões e outros veículos aéreos, a turboélice, multimotores, de peso inferior ou igual a 7.000 kg, vazios",2068000
332,Zona Não Declarada,2019,6,Mudas de plantas ornamentais orquídeas,935915
333,Zona Não Declarada,2019,7,"Aviões e outros veículos aéreos, 7 toneladas < peso <= 15 toneladas, vazios, a turboélice",3199882
334,Zona Não Declarada,2019,8,"Aviões e outros veículos aéreos, 7 toneladas < peso <= 15 toneladas, vazios, a turboélice",4634397


5.32 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Posição de cada produto no ranking de valor transacionado por estado, por ano

In [29]:
%%timeit -n1 -r1
query = f'''
SELECT estado,
       ano,
       produto,
       total,
       ROW_NUMBER() OVER(PARTITION BY estado, ano
                         ORDER BY total DESC) AS POSITION
FROM
  (SELECT SG_UF_NCM estado,
          strftime("%Y", k.DATA) ano,
          k.CO_NCM produto,
          SUM(VL_FOB) total
   FROM {kind} k
   GROUP BY ano,
            estado,
            produto)
'''
display(pd.read_sql(query, conn))

,estado,ano,produto,total,POSITION
0,AC,2017,7032090,393150,1
1,AC,2017,39041090,293144,2
2,AC,2017,8012200,235580,3
3,AC,2017,28332960,168608,4
4,AC,2017,84385000,123502,5
...,...,...,...,...,...
202291,ZN,2019,85366100,3,784
202292,ZN,2019,84603900,2,785
202293,ZN,2019,85182990,2,786
202294,ZN,2019,96138000,2,787


8.41 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


 ## Top 3 produtos com maior valor transacionado de 2017 a 2019, por ano, por estado

In [30]:
%%timeit -n1 -r1
# Inspired by https://stackoverflow.com/a/3800572
# See https://www.sqlite.org/windowfunctions.html
query = f'''
WITH totals AS
  (SELECT estado,
          ano,
          produto,
          total,
          ROW_NUMBER() OVER(PARTITION BY estado, ano
                            ORDER BY total DESC) AS POSITION
   FROM
     (SELECT SG_UF_NCM estado,
             strftime("%Y", k.DATA) ano,
             k.CO_NCM produto,
             SUM(VL_FOB) total
      FROM {kind} k
      GROUP BY ano,
               estado,
               produto))
SELECT u.NO_UF estado,
       t.ano,
       n.NO_NCM_POR produto,
       t.total
FROM totals t
JOIN ncm n ON produto=n.CO_NCM
JOIN uf u ON estado=u.SG_UF
WHERE t.position <= 3
ORDER BY estado, ano, total DESC
'''
display(pd.read_sql(query, conn))

,estado,ano,produto,total
0,Acre,2017,"Alhos, frescos ou refrigerados, exceto para semeadura",393150
1,Acre,2017,"Outros policloretos de vinila, em formas primárias",293144
2,Acre,2017,"Castanha-do-pará, fresca ou seca, sem casca",235580
3,Acre,2018,"Revestimentos para pisos (pavimentos) e capachos, de borracha vulcanizada não endurecida",1031956
4,Acre,2018,Outros adesivos à base de plásticos,328032
...,...,...,...,...
247,Zona Não Declarada,2018,Mudas de plantas ornamentais orquídeas,12126258
248,Zona Não Declarada,2018,"Outros medicamentos com compostos heterocíclicos, etc, em doses",10728822
249,Zona Não Declarada,2019,"Aviões e outros veículos aéreos, 7 toneladas < peso <= 15 toneladas, vazios, a turboélice",31682883
250,Zona Não Declarada,2019,Mudas de plantas ornamentais orquídeas,11840675


7.89 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Top 3 produtos com maior valor transacionado em 2019, por mês, por estado

In [31]:
%%timeit -n1 -r1
year = 2019
query = f'''
WITH totals AS
  (SELECT estado,
          ano,
          mês,
          produto,
          total,
          ROW_NUMBER() OVER(PARTITION BY estado, mês
                            ORDER BY total DESC) AS POSITION
   FROM
     (SELECT SG_UF_NCM estado,
             strftime("%Y", k.DATA) ano,
             ltrim(strftime("%m", k.DATA), "0") mês,
             k.CO_NCM produto,
             SUM(VL_FOB) total
      FROM {kind} k
      WHERE ano = "{year}"
      GROUP BY mês,
               estado,
               produto))
SELECT u.NO_UF estado,
       t.ano,
       t.mês,
       n.NO_NCM_POR produto,
       t.total
FROM totals t
JOIN ncm n ON produto=n.CO_NCM
JOIN uf u ON estado=u.SG_UF
WHERE t.position <= 3
ORDER BY estado, mês, total DESC
'''
display(pd.read_sql(query, conn))

,estado,ano,mês,produto,total
0,Acre,2019,1,"Pneumáticos novos, de borracha, dos tipos utilizados em automóveis de passageiros (incluindo os veículos de uso misto (station wagons) e os automóveis de corrida)",118213
1,Acre,2019,1,"Outros aparelhos elevadores ou transportadores, de ação contínua, para mercadorias",85945
2,Acre,2019,1,"Outros policloretos de vinila, em formas primárias",47423
3,Acre,2019,10,"Outros produtos imunológicos, apresentados em doses ou acondicionados para venda a retalho",43399
4,Acre,2019,10,"Outros policloretos de vinila, em formas primárias",25750
...,...,...,...,...,...
1003,Zona Não Declarada,2019,8,"Produções originais de arte estatuária ou de escultura, de quaisquer matérias",1503620
1004,Zona Não Declarada,2019,8,Outras colheitadeiras de algodão,1291653
1005,Zona Não Declarada,2019,9,"Aviões e outros veículos aéreos, 7 toneladas < peso <= 15 toneladas, vazios, a turboélice",3964705
1006,Zona Não Declarada,2019,9,Mudas de plantas ornamentais orquídeas,746433


5.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Valor FOB total transacionado em 2019

In [32]:
%%timeit -n1 -r1
year = 2019
query = f'''
SELECT strftime("%Y", DATA) ano,
       SUM(VL_FOB) total
FROM {kind}
WHERE ano = "{year}"
'''
display(pd.read_sql(query, conn))

,ano,total
0,2019,177347934749


1.46 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Percentual com que cada estado contribuiu para o total transacionado pelo Brasil em 2019

In [33]:
%%timeit -n1 -r1
year=2019
query = f'''
SELECT strftime("%Y", k.DATA) ano,
       u.NO_UF,
       SUM(VL_FOB) total,
       100.0 * SUM(VL_FOB) / (SELECT SUM(VL_FOB)
                FROM {kind}
                WHERE strftime("%Y", DATA) = "{year}") percentual
FROM {kind} k
JOIN uf u ON SG_UF_NCM=u.SG_UF
WHERE ano="{year}"
GROUP BY SG_UF_NCM
ORDER BY total DESC
'''
display(pd.read_sql(query, conn))

,ano,NO_UF,total,percentual
0,2019,São Paulo,59344945286,33.462439
1,2019,Rio de Janeiro,21207242422,11.957987
2,2019,Santa Catarina,16920808903,9.541024
3,2019,Paraná,12695474202,7.158513
4,2019,Amazonas,10163473885,5.730810
5,2019,Rio Grande do Sul,9895877749,5.579923
6,2019,Minas Gerais,8954990584,5.049391
7,2019,Bahia,6772304966,3.818655
8,2019,Espírito Santo,6274742006,3.538097
9,2019,Pernambuco,5091997829,2.871191


3.86 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
